In [76]:
# Imports 
import numpy as np
from glob import glob


In [77]:
# Load data
data_dir = 'flower_data'
train_dir = data_dir + '/train/*/*'
valid_dir = data_dir + '/valid/*/*'

# load filenames for flowers images
flowers_train = np.array(glob(train_dir))
flowers_valid = np.array(glob(valid_dir))

# print number of images in each dataset
print('There are %d total flowers train images.' % len(flowers_train))
print('There are %d total flowers valid images.' % len(flowers_valid))


There are 6552 total flowers train images.
There are 818 total flowers valid images.
